In [1]:
# generate_transactions_2025.py
# Generates transactions_2025_OBIE_vfinal.xlsx with three sheets:
# - transactions_2025
# - bank_accounts
# - mcc_codes
#
# Dependencies: pandas, numpy, openpyxl
# Install: pip install pandas numpy openpyxl

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

#Set fixed starting point, so that entire data-generation process is fully reproducible.
random.seed(42)
np.random.seed(42)

n_transactions = 1000

# --- Bank accounts ---
bank_accounts = pd.DataFrame([
    {"account_id": "A001", "account_name": "Barclays Current Account", "bank_name": "Barclays", "currency": "GBP", "account_type": "current", "sort_code":"20-00-00", "masked_iban":"GB11BARC20000012345678"},
    {"account_id": "A002", "account_name": "Revolut Spending Account", "bank_name": "Revolut", "currency": "EUR", "account_type": "current", "sort_code":"40-00-00", "masked_iban":"GB22REVX40000098765432"},
    {"account_id": "A003", "account_name": "Chase Credit Card", "bank_name": "Chase", "currency": "USD", "account_type": "credit_card", "sort_code":"00-00-00", "masked_iban":"US00CHAS00000055554444"}
])

# --- MCC table ---
mcc_list = [
    ("Starbucks", 5812, "Food & Drink"),
    ("McDonalds", 5812, "Food & Drink"),
    ("Nando's", 5812, "Food & Drink"),
    ("Chipotle", 5812, "Food & Drink"),
    ("Uber", 4121, "Transport"),
    ("Lyft", 4121, "Transport"),
    ("Shell Fuel", 5541, "Fuel"),
    ("Trainline", 4111, "Transport"),
    ("Tesco", 5411, "Groceries"),
    ("Walmart", 5411, "Groceries"),
    ("Lidl", 5411, "Groceries"),
    ("Whole Foods", 5411, "Groceries"),
    ("Netflix", 4899, "Streaming / Subscriptions"),
    ("Spotify", 4899, "Streaming / Subscriptions"),
    ("Apple TV", 4899, "Streaming / Subscriptions"),
    ("Adobe", 5734, "Software / Subscriptions"),
    ("Gymbox", 7997, "Fitness"),
    ("Amazon", 5942, "Online Shopping"),
    ("Zara", 5651, "Clothing"),
    ("IKEA", 5712, "Home Furnishing"),
    ("Target", 5311, "Retail"),
    ("PAYROLL LTD", 6011, "Income"),
    ("Direct Deposit", 6011, "Income"),
    ("Upwork", 6011, "Income"),
    ("PayPal Transfer", 6011, "Income"),
    ("Interest Credit", 6011, "Income"),
    ("ETF Dividend", 6011, "Income"),
    ("Amazon Refund", 6011, "Income"),
    ("Expense Reimb.", 6011, "Income"),
    ("From Revolut", 6011, "Income"),
    ("Incoming Transfer", 6011, "Income"),
    ("LANDLORD LTD", 6513, "Rent / Mortgage"),
    ("RENT PAYMENT", 6513, "Rent / Mortgage"),
    ("Electricity Co", 4900, "Utilities"),
    ("Water Bill", 4900, "Utilities"),
    ("Internet", 4899, "Utilities"),
    ("Health Insurance", 6300, "Insurance"),
    ("Car Insurance", 6300, "Insurance"),
    ("Verizon", 4812, "Mobile / Telecom"),
    ("EE", 4812, "Mobile / Telecom"),
    ("Vodafone", 4812, "Mobile / Telecom"),
    ("Hair Salon", 7230, "Personal Care"),
    ("Barber", 7230, "Personal Care"),
    ("Pharmacy", 5912, "Personal Care"),
    ("Pending – Starbucks", 5812, "Pending"),
    ("Reversal – Uber", 4121, "Reversal"),
    ("Overdraft Fee", 6012, "Bank Fees"),
    ("Credit Card Payment", 6012, "Financial"),
    ("****", 0, "Corrupted"),
    (None, 0, "Missing")
]
mcc_codes = pd.DataFrame(mcc_list, columns=["merchant_name", "mcc_code", "category"])

# Function to create date string errors
def mixed_date_string(dt):
    fmt = random.choices(
        ["%Y-%m-%d %H:%M:%S", "%d/%m/%Y %H:%M", "%Y/%m/%d %H:%M:%S", "%d-%m-%Y %H:%M:%S", "%b %d, %Y %I:%M %p"],
        weights=[0.5, 0.15, 0.15, 0.1, 0.1]
    )[0]
    return dt.strftime(fmt)

# Seasonality weights (higher Nov-Dec; summer travel months higher)
month_weights = {
    1:8, 2:7, 3:6, 4:7, 5:7, 6:8, 7:9, 8:9, 9:6, 10:7, 11:12, 12:14
}

statuses = ["Booked", "Pending", "Failed"]
transaction_types = ["Credit", "Debit", "Transfer", "Refund"]




#Add: Random letter drops (common OCR/API error)
def drop_random_char(s):
    if len(s) <= 3:
        return s
    idx = random.randint(0, len(s)-1)
    return s[:idx] + s[idx+1:]
#Add: Random letter swaps (typo)
def swap_adjacent_chars(s):
    if len(s) <= 3:
        return s
    idx = random.randint(0, len(s)-2)
    return s[:idx] + s[idx+1] + s[idx] + s[idx+2:]
#Add: Duplicate characters (fat-finger mistakes)
def duplicate_random_char(s):
    idx = random.randint(0, len(s)-1)
    return s[:idx] + s[idx] + s[idx:] 

rows = []
for _ in range(n_transactions):
    acc = bank_accounts.sample(1).iloc[0]
    merchant_row = mcc_codes.sample(1).iloc[0]
    merchant = merchant_row["merchant_name"]
    mcc = merchant_row["mcc_code"]
    category = merchant_row["category"]
    # determine month with weights
    months = list(month_weights.keys())
    weights = list(month_weights.values())
    month = random.choices(months, weights=weights, k=1)[0]
    day = random.randint(1,28)
    hour = random.randint(6,22)
    minute = random.randint(0,59)
    second = random.randint(0,59)
    booking_dt = datetime(2025, month, day, hour, minute, second)
    value_dt = booking_dt + timedelta(days=random.choice([0,0,1]))

    # amount logic
    if category in ["Rent / Mortgage"]:
        amount = -random.choice([800,1000,1200,1300])
    elif category in ["Income"]:
        amount = random.choice([1500,2000,2500])
    elif category in ["Utilities","Insurance","Mobile","Bank Fees"]:
        amount = -round(random.uniform(20,400),2)
    elif category in ["Food & Drink","Groceries","Personal Care","Transport","Fuel"]:
        amount = -round(random.uniform(2,120),2)
    elif category in ["Online Shopping","Retail","Clothing","Home Furnishing","Streaming / Subscriptions","Fitness"]:
        amount = -round(random.uniform(5,800),2)
    else:
        amount = -round(random.uniform(5,200),2)

    # overdraft & credit card behavior
    # 2% of the time make the transaction an overdraft fee
    if random.random() < 0.02:
        merchant = "Overdraft Fee"
        mcc = 6012
        category = "Bank Fees"
        amount = -random.choice([20,35,50])
    # Transactions from the credit card account are capped at 6% (85% Credit card use and 15% Repayment)
    if acc["account_type"] == "credit_card" and random.random() < 0.06:
        if random.random() < 0.85:
            amount = -abs(round(random.uniform(5,1000),2))
        else:
            merchant = "Credit Card Payment"
            mcc = 6012
            category = "Financial"
            amount = random.choice([200,500,1000])

    # errors
    errors = []
    # wrong sign (flip) sometimes
    if random.random() < 0.05:
        amount = None if amount is None else -amount
        errors.append("wrong_sign")
    # missing amount
    if random.random() < 0.03:
        amount = None
        errors.append("missing_amount")
    # corrupted merchant
    merchant_display = merchant
    if random.random() < 0.05:
        merchant_display = "****"
        errors.append("corrupted_merchant")
    else:
        # casing and extra spaces
        if merchant_display is not None:
            if random.random() < 0.2:
                merchant_display = merchant_display.lower()
            if random.random() < 0.2:
                merchant_display = " " + merchant_display + " "
            if random.random() < 0.05:
                merchant_display = drop_random_char(merchant_display)
            if random.random() < 0.05:
                merchant_display = swap_adjacent_chars(merchant_display)
            if random.random() < 0.05:
                merchant_display = duplicate_random_char(merchant_display)

    # missing merchant
    if random.random() < 0.02:
        merchant_display = None
        errors.append("missing_merchant")
    # mixed currency / exchange simulation
    original_currency = acc["currency"]
    currency = acc["currency"]
    exchange_rate = None
    if random.random() < 0.08:
        currency = random.choice(["GBP","EUR","USD"])
        if currency != acc["currency"]:
            exchange_rate = round(random.uniform(0.6,1.5),4)
            original_currency = acc["currency"]

    status = random.choices(statuses, weights=[0.8,0.15,0.05])[0]
    if status == "Pending":
        errors.append("pending_authorization")
    if random.random() < 0.02:
        merchant_display = "Reversal – " + (merchant if merchant else "UNKNOWN")
        errors.append("reversal")
        amount = None if amount is None else -amount

    raw_date = mixed_date_string(booking_dt) if random.random() < 0.25 else None

    reference = f"REF{random.randint(10000,99999)}"
    bank_tx_code = f"BTC{random.randint(100,999)}"
    prop_bank_tx_code = f"PBC{random.randint(100,999)}"
    transaction_id_obie = f"O{random.randint(1000000,9999999)}"

    counterparty_name = None
    counterparty_account = None
    if random.random() < 0.08:
        counterparty_name = f"Counterparty {random.randint(1,200)}"
        counterparty_account = f"CP{random.randint(100000,999999)}"

    row = {
        "AccountId": acc["account_id"],
        "TransactionId": f"T{random.randint(100000,999999)}",
        "TransactionId_OBIE": transaction_id_obie,
        "BookingDateTime": booking_dt,
        "ValueDateTime": value_dt,
        "RawDateString": raw_date,
        "InstructedAmount": amount if amount is not None else round(random.uniform(5,2500),2),
        "TransactionAmount": amount,
        "Currency": currency,
        "OriginalCurrency": original_currency,
        "ExchangeRate": exchange_rate,
        # Can update so only credit card account transactions are counted as credit
        "TransactionType": random.choice(transaction_types),
        "MerchantDetails": (f"{merchant_display} - details" if merchant_display else None),
        "MerchantName": merchant_display,
        "MerchantCategoryCode": mcc,
        "BankTransactionCode": bank_tx_code,
        "ProprietaryBankTransactionCode": prop_bank_tx_code,
        "Reference": reference,
        "Balance": round(random.uniform(-1000,20000),2),
        "CounterpartyName": counterparty_name,
        "CounterpartyAccount": counterparty_account,
        "Status": status,
        "Category": category,
        "AccountCurrency": acc["currency"],
        "Errors": ";".join(errors) if errors else ""
    }
    rows.append(row)

df_tx = pd.DataFrame(rows)

# Add duplicates (~5%)
dupes = df_tx.sample(frac=0.05, random_state=1)
# Combine with unique and shuffle rows
df_tx = pd.concat([df_tx, dupes], ignore_index=True).sample(frac=1, random_state=2).reset_index(drop=True)

# Save workbook
excel_path = r"C:\Users\GeoffreyRwamakuba\Downloads\Tdata2.xlsx"
with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    df_tx.to_excel(writer, sheet_name="transactions_2025", index=False)
    bank_accounts.to_excel(writer, sheet_name="bank_accounts", index=False)
    mcc_codes.to_excel(writer, sheet_name="mcc_codes", index=False)

print("Wrote", excel_path)

Wrote C:\Users\GeoffreyRwamakuba\Downloads\Tdata2.xlsx
